In [4]:
import glob
import pandas as pd
import numpy as np
import os
from IPython.display import HTML
import scipy

In [5]:
!ls ../../ppc/scripts

autotune_12000.sh
autotune_eval_10000.sh
autotune_eval_12000_seed.sh
autotune_eval_12000.sh
autotune_eval_8000.sh
autotune_eval_zifa_A.sh
autotune_eval_zifa_B.sh
autotune_eval_zifa_C_new.sh
autotune_eval_zifa_C.sh
autotune_eval_zifa_D_new.sh
autotune_eval_zifa_D.sh
autotune_eval_zifa_F_new.sh
autotune_eval_zifa_F.sh
autotune_eval_zifa_F_sos10.sh
autotune_eval_zifa_G_new.sh
autotune_eval_zifa_G.sh
autotune_eval_zifa_H.sh
autotune_eval_zifa_I.sh
autotune.sh
autotune_zi_12000.sh
autotune_zifa_new.sh
autotune_zifa.sh
autotune_zi.sh
aws_sos
brain_small
brain_small_old
brain_small.zip
catchup_new.txt
catchup.txt
check_hemato.py
cortex
cortex_bis
cortex_old
cortex.zip
data
hemato
hemato_old
hemato_zinb_1200_50_results.json
hemato_zinb_1200_50_results_new.json
hemato.zip
klein_dataset
klein_dataset_nb_1200_50_results.json
klein_dataset_zinb_1200_50_results.json
log_poisson_nb_dataset_10000
log_poisson_nb_dataset_10000_old1
log_poisson_nb_dataset_10000_sim1
log_poisson_nb_dataset_12000
log_pois

In [6]:



data_path = '../../ppc/scripts'
dataset_names = {'cortex': 'Cortex',
                 'hemato': 'Hemato',
                 'brain_small': 'Brain Small',
                 'pbmc': 'PBMC',
                 'retina': 'Retina'}

data_files = {}
data_names = {}
for dataset_name in dataset_names:
    data_files[dataset_name] = sorted(glob.glob(os.path.join(data_path, '{}/*.p'.format(dataset_name))))
    data_names[dataset_name] = []
    for data_file in data_files[dataset_name]:
        data_name = data_file.split('/')[-1].split('_')[0]
        data_names[dataset_name].append(data_name)

for dataset_name in list(dataset_names.keys()):
    if len(data_names[dataset_name]) != 2:
        del dataset_names[dataset_name]
        del data_files[dataset_name]
        del data_names[dataset_name]

In [7]:
dataset_names

{'cortex': 'Cortex',
 'hemato': 'Hemato',
 'brain_small': 'Brain Small',
 'pbmc': 'PBMC',
 'retina': 'Retina'}

In [8]:
data_names

{'cortex': ['nb', 'zinb'],
 'hemato': ['nb', 'zinb'],
 'brain_small': ['nb', 'zinb'],
 'pbmc': ['nb', 'zinb'],
 'retina': ['nb', 'zinb']}

In [9]:
df = {}
for dataset_name in dataset_names:
    dfs = []
    for data_name, f in zip(data_names[dataset_name], data_files[dataset_name]):
        if data_name != 'zifa_full':

            #my_df = pd.read_csv(f, sep='\t')
            my_df = pd.read_pickle(f)
            my_df.loc[:, 'model'] = data_name
            dfs.append(my_df)
            print(data_name, f)
    #         display(my_df)
    df[dataset_name] = pd.concat(dfs, axis=0)

nb ../../ppc/scripts/cortex/nb_cortex.p
zinb ../../ppc/scripts/cortex/zinb_cortex.p
nb ../../ppc/scripts/hemato/nb_hemato.p


/home/oscar/anaconda3/envs/scvienv/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


zinb ../../ppc/scripts/hemato/zinb_hemato.p
nb ../../ppc/scripts/brain_small/nb_brain_small.p


/home/oscar/anaconda3/envs/scvienv/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


zinb ../../ppc/scripts/brain_small/zinb_brain_small.p


/home/oscar/anaconda3/envs/scvienv/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


nb ../../ppc/scripts/pbmc/nb_pbmc.p
zinb ../../ppc/scripts/pbmc/zinb_pbmc.p
nb ../../ppc/scripts/retina/nb_retina.p
zinb ../../ppc/scripts/retina/zinb_retina.p


/home/oscar/anaconda3/envs/scvienv/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]
/home/oscar/anaconda3/envs/scvienv/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


In [10]:
from statsmodels.stats.weightstats import ttest_ind

metrics = ['ll_ll', 't_dropout_ks_stat', 't_ratio_ks_stat', 't_cv_ks_stat']

h1_hypothesis = ['greater', 'greater', 'greater', 'greater']


df_summary = {}

n_to_keep = 50

for dataset_name in dataset_names:

    df_nb = df[dataset_name].loc[df[dataset_name].model=='nb', metrics].sort_values(by='ll_ll').iloc[:n_to_keep,:]
    df_zinb = df[dataset_name].loc[df[dataset_name].model=='zinb', metrics].sort_values(by='ll_ll').iloc[:n_to_keep,:]
    
    df_nb_2 = df_nb.copy()
    df_nb_2['model'] = 'nb'
    df_zinb_2 = df_zinb.copy()
    df_zinb_2['model'] = 'zinb'
    df_new = pd.concat([df_nb_2, df_zinb_2])
    
    print("Shapes")
    print(df_nb.shape)
    print(df_zinb.shape)
    print(df_new.shape)
    print(df_new)

    def get_pvals(gby, other_df):
        my_df = gby[metrics]
        print(gby)
        assert my_df.shape[1] == len(metrics)
        assert other_df.shape[1] == len(metrics)
        pvals = []
        for idx, alternative in enumerate(h1_hypothesis):
            assert len(other_df.values[:, idx]) != len(h1_hypothesis), (len(other_df.values[:, idx]), len(h1_hypothesis))
            #_, pval, _ = ttest_ind(other_df.values[:, idx], my_df.values[:, idx], alternative=alternative)
            try:
                _, pval = scipy.stats.mannwhitneyu(np.sort(other_df.values[:, idx])[:n_to_keep],
                                       np.sort(my_df.values[:, idx])[:n_to_keep][::-1],
                                       use_continuity=True,
                                       alternative=alternative)
            except ValueError:
                pval = 0.5
            
            pvals.append(pval)
        return np.array(pvals)

    pvals_against_zinb = df_new.groupby('model').apply(get_pvals, other_df=df_zinb)
    pvals_against_zinb = (pvals_against_zinb
             .apply(lambda x: pd.Series(x))
             .T)
    pvals_against_zinb = pvals_against_zinb.rename(index={idx: met for (idx,met) in enumerate(metrics)})

    pvals_against_nb = df_new.groupby('model').apply(get_pvals, other_df=df_nb)
    pvals_against_nb = (pvals_against_nb
             .apply(lambda x: pd.Series(x))
             .T)
    pvals_against_nb = pvals_against_nb.rename(index={idx: met for (idx,met) in enumerate(metrics)})

    def get_summary(gby):
        res = {}
        res['median'] = gby.median()
        res['pvals_against_nb'] = pvals_against_nb[gby.name]
        res['pvals_against_zi'] = pvals_against_zinb[gby.name]
        return pd.DataFrame(res).T

    # df_summary = df.groupby('model')['ll_ll', 'imputation_median_imputation_score', 't_dropout_ks_stat',
    #                                  't_ratio_ks_stat', 't_cv_ks_stat'].agg(['mean', 'std']).T
    df_summary[dataset_name] = df[dataset_name].groupby('model')[metrics].apply(get_summary)


    df_summary[dataset_name] = df_summary[dataset_name].stack().unstack(1).sort_index(level=1).swaplevel()

Shapes
(50, 4)
(50, 4)
(100, 5)
          ll_ll  t_dropout_ks_stat  t_ratio_ks_stat  t_cv_ks_stat model
21  1255.358842           0.034050         0.037634      0.229391    nb
4   1256.762258           0.064516         0.077061      0.207885    nb
6   1257.202995           0.059140         0.048387      0.189964    nb
14  1257.524477           0.044803         0.077061      0.186380    nb
19  1257.880044           0.053763         0.043011      0.200717    nb
7   1259.214564           0.048387         0.051971      0.229391    nb
36  1259.420198           0.043011         0.075269      0.182796    nb
43  1260.844621           0.044803         0.046595      0.177419    nb
1   1261.181949           0.032258         0.039427      0.220430    nb
40  1261.319039           0.102151         0.089606      0.189964    nb
10  1262.261907           0.039427         0.039427      0.197133    nb
23  1262.368929           0.039427         0.084229      0.181004    nb
42  1262.715630           0.0698

Shapes
(50, 4)
(50, 4)
(100, 5)
          ll_ll  t_dropout_ks_stat  t_ratio_ks_stat  t_cv_ks_stat model
28  1013.607896           0.026667         0.030833      0.032500    nb
41  1013.629078           0.021667         0.027500      0.043333    nb
40  1013.650833           0.040833         0.061667      0.069167    nb
3   1013.889262           0.042500         0.054167      0.066667    nb
22  1013.913212           0.021667         0.046667      0.043333    nb
11  1013.986690           0.039167         0.069167      0.034167    nb
20  1014.002722           0.015833         0.037500      0.059167    nb
36  1014.081080           0.049167         0.065833      0.015833    nb
33  1014.137755           0.058333         0.096667      0.070000    nb
43  1014.141984           0.019167         0.038333      0.043333    nb
37  1014.177627           0.045833         0.061667      0.064167    nb
48  1014.198102           0.018333         0.047500      0.058333    nb
7   1014.204155           0.0483

# Export to Latex

In [11]:
def boldizer(row):
    to_bold = (row.pvals_against_nb <= 5e-2) | (row.pvals_against_zi <= 5e-2)
#     print(to_bold)
    if to_bold:
        row.latex = '\mathbf{' + row.latex + '}'
    return row

In [12]:
n_dec = 3

work_df = {}

for dataset_name in dataset_names:

    work_df[dataset_name] = (df_summary[dataset_name]
               .reset_index(level=-1)
               .reset_index()
               .assign(latex=lambda x: x['median'].round(n_dec).astype(str))
               .apply(boldizer, axis=1)
               .assign(Metric=lambda x: 
                       x.loc[:, 'index'].replace({
                           'll_ll': 'Negative LL',
                           't_cv_ks_stat': 'Coefficient of variation',
                           't_dropout_ks_stat': 'Dropout ratio',
                           't_ratio_ks_stat': 'Zeros-to-expression ratio',
                           'model': 'Model'
               }),
                       latex=lambda x: '$' + x.latex + '$'
                      )
              )

    work_df[dataset_name] = work_df[dataset_name].rename(columns={'model': 'Model', 'index': 'metric',
                                                                 'Metric': 'Metric (medians)'})
    work_df[dataset_name]['Model'] = work_df[dataset_name]['Model'].str.upper()
    #work_df[dataset_name]['Dataset'] = dataset_names[dataset_name]
    

In [13]:
pivot = {}
for dataset_name in dataset_names:
    pivot[dataset_name] = pd.pivot(work_df[dataset_name], index='Metric (medians)', values='latex', columns='Model')
    pivot[dataset_name]['Dataset'] = dataset_names[dataset_name]
    pivot[dataset_name].reset_index(inplace=True)
    pivot[dataset_name].set_index(['Dataset', 'Metric (medians)'], inplace=True)
pivot = pd.concat(list(pivot.values()), axis=0)

In [14]:
pivot.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 20 entries, (Cortex, Coefficient of variation) to (Retina, Zeros-to-expression ratio)
Data columns (total 2 columns):
NB      20 non-null object
ZINB    20 non-null object
dtypes: object(2)
memory usage: 553.0+ bytes


In [15]:
res = pivot.to_latex(escape=False)
print(res)

\begin{tabular}{llll}
\toprule
       & Model &                   NB &                 ZINB \\
Dataset & Metric (medians) &                      &                      \\
\midrule
Cortex & Coefficient of variation &     $\mathbf{0.207}$ &              $0.235$ \\
       & Dropout ratio &              $0.081$ &     $\mathbf{0.047}$ \\
       & Negative LL &  $\mathbf{1270.909}$ &           $1278.183$ \\
       & Zeros-to-expression ratio &              $0.098$ &              $0.097$ \\
Hemato & Coefficient of variation &               $0.19$ &     $\mathbf{0.151}$ \\
       & Dropout ratio &              $0.155$ &              $0.122$ \\
       & Negative LL &            $911.455$ &   $\mathbf{904.807}$ \\
       & Zeros-to-expression ratio &              $0.252$ &              $0.323$ \\
Brain Small & Coefficient of variation &     $\mathbf{0.052}$ &              $0.071$ \\
       & Dropout ratio &              $0.039$ &              $0.044$ \\
       & Negative LL &           $1975.474